# Final Assignment for Applied Data Science

### Intoduction

In our days there is a trend to travel overseas to enjoy new food, as a food 
enthusiastic I would like to know if a certain city has similar cuisines as in
my country. Usually this requires me to go and search the web for restaurants in the 
country that i'm traveling to.
I will try to tackle the problem using the Foursquare data to figure the similarity
in cuisines between two cities.

### Data

The data that i will get from the API will give me the restaurants in the area 
that i'm looking to travel.  
For each of the restaurants i will extract the category of the restaurant and location.

In [5]:
import pandas as pd
pd.read_csv('Chicago.csv')[['Name', 'Category', 'Latitude', 'Longitude']].head()

,Name,Category,Latitude,Longitude
0,Bari,Sandwiches,41.891178,-87.655628
1,Lost Larson Bakery,Bakery,41.978617,-87.668411
2,Publican Quality Meats,Deli / Bodega,41.886642,-87.648718
3,"Joe's Seafood, Prime Steak & Stone Crab",Seafood,41.891828,-87.625444
4,Weber's Bakery,Bakery,41.791695,-87.798656


Using this table i will see the percentage of each category in this area (the range will be 0-1).

After getting the data for each city i will intersect the tables and get a list of similar cuisines.
I will add the percentages in the list according to the table of the city that i'm comparing to.

The closer we get to 1.0 the similar the cities are.